In [ ]:
# Based on code from the following link:
# https://towardsdatascience.com/multi-label-classification-using-bert-roberta-xlnet-xlm-and-distilbert-with-simple-transformers-b3e0cda12ce5

In [ ]:
#!pip install simpletransformers==0.34.4

In [ ]:
# import keras
# keras.__version__

In [ ]:
# import tensorflow
# tensorflow.__version__

In [ ]:
# import torch
# torch.__version__

In [ ]:
# !jupyter --version

In [ ]:
# !pip install scikit-learn==0.23.1

In [ ]:
# !pip install pandas==1.0.5

In [ ]:
# !pip show simpletransformers 

In [ ]:
# import sklearn
# sklearn.__version__

In [1]:
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel
#import ast

C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\backend\utils.py:88: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
#from simpletransformers.experimental.classification import MultiLabelClassificationModel

C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\backend\utils.py:88: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


# Model Training

In [2]:
df = pd.read_csv('train21Aug27.tsv', sep='\t')
#df = pd.read_csv('../Mycode/mimic3_preprocess/train.tsv', sep='\t')
df= df.sample(frac=0.2, random_state=57)
df.head()

,text,labels
252207,Newborn Med Attending\n\nAsked by Dr [**Last N...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
185389,[**2125-2-13**] 1:07 AM\n CT HEAD W/O CONTRAST...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299280,[**2145-4-5**] 11:34 AM\n CHEST (PORTABLE AP) ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
62385,TITLE: Respiratory Care:\n Patient extubated...,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
73962,"Neonatology Attending\nDOL#32, CGA 30 [**1-9**...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
df.labels = df.labels.apply(eval)
df.labels.shape

(86494,)

In [5]:
df.shape

(86494, 2)

In [8]:
#ab= df['labels'][0:1]

1


In [10]:
model = MultiLabelClassificationModel('bert', "emilyalsentzer/Bio_ClinicalBERT", num_labels=101, 
                                      args={'train_batch_size':8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 5e-5,
                                            'num_train_epochs': 2,
                                            'max_seq_length': 250,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'save_optimizer_and_scheduler': True})

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassifica

In [11]:
model.train_model(df)

Epochs 0/2. Running Loss:    0.2371: 100%|██████████| 10812/10812 [34:54<00:00,  5.16it/s]
C:\Miniconda\envs\py38tfpt\lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
Epoch 2 of 2: 100%|██████████| 2/2 [1:08:42<00:00, 2061.18s/it]


(1350, 0.21466032622244072)

# Sliding Window Experimental Model Training

import pandas as pd
#from simpletransformers.classification import MultiLabelClassificationModel
import ast
#from simpletransformers.experimental.classification import MultiLabelClassificationModel

df = pd.read_csv('./train.tsv', sep='\t')
df.labels = df.labels.apply(eval)

model = MultiLabelClassificationModel('bert', "emilyalsentzer/Bio_ClinicalBERT", num_labels=101, 
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 5e-5,
                                            'num_train_epochs': 2,
                                            'max_seq_length': 150,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           #'tensorboard_dir':'./outputs/',
                                           #'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

model.train_model(df)

# Model Evaluation

In [2]:
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel
import ast
#from simpletransformers.experimental.classification import MultiLabelClassificationModel
import sklearn
import numpy as np

C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Miniconda\envs\py38tfpt\lib\site-packages\torchaudio\backend\utils.py:88: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


model = MultiLabelClassificationModel('bert', './outputs/checkpoint-386-epoch-2', num_labels=101,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           #'tensorboard_dir':'./outputs/',
                                           #'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

In [5]:
model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-1350-epoch-2", num_labels=101, 
                                      args={'train_batch_size':8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 5e-5,
                                            'num_train_epochs': 2,
                                            'max_seq_length': 250,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'save_optimizer_and_scheduler': True})

In [6]:

eval_df = pd.read_csv('test21Aug27.tsv', sep='\t')
eval_df= eval_df.sample(frac=0.2, random_state=57, ignore_index=True)
eval_df.labels = eval_df.labels.apply(eval)
print(eval_df.shape)

(10812, 2)


In [27]:
print(eval_df.tail())

                                                   text  \
5401  TITLE:\n   Chief Complaint:\n   Pulmonary\n   ...   
5402  88yo woman w hx of HTN, hypothyroidism, hyperl...   
5403  Respiratory Care\nBaby continues on prong CPAP...   
5404  Neonatology Attending Note\n\nInfant well-know...   
5405  Neonatology NP Note\nPLease refer to attending...   

                                                 labels  
5401  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5402  [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5403  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5404  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5405  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [7]:

#model.get_named_parameters()
predictions, raw_outputs, wrong_pred = model.eval_model(eval_df)


Running Evaluation: 100%|██████████| 1352/1352 [01:38<00:00, 13.77it/s]


In [15]:
print(raw_outputs)

[[0.06524658 0.14440918 0.10821533 ... 0.01751709 0.07922363 0.05633545]
 [0.08282471 0.19348145 0.12744141 ... 0.01757812 0.08959961 0.06994629]
 [0.02700806 0.02975464 0.04647827 ... 0.13708496 0.0411377  0.03442383]
 ...
 [0.06695557 0.15319824 0.10931396 ... 0.016922   0.07922363 0.05697632]
 [0.03044128 0.03216553 0.05072021 ... 0.15539551 0.0440979  0.03759766]
 [0.1427002  0.27563477 0.18457031 ... 0.02423096 0.12023926 0.09967041]]


In [8]:

y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1
#print(y_true)
print(f"y_true: {y_true.shape}")

#m_outputs = raw_outputs[0:10]
for cut_off in np.linspace(0.1,1,9,endpoint=False):  
    y_pred = np.empty((0,101))
    for i in range(len(raw_outputs)):
        m_output = np.where (raw_outputs[i]>cut_off, 1,0) 
        y_pred = np.vstack([y_pred,m_output])
    
    print(f"y_true shape: {y_true.shape}  y_pred Shape: {y_pred.shape}")
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    #print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))

y_true: (10812, 101)
y_true shape: (10812, 101)  y_pred Shape: (10812, 101)


Calculated F-scores for cutoff: 0.1

Macro: 0.23734861068623364
Micro: 0.3771816127538286
Weighted: 0.4084807902050138
Samples: 0.3736207804009601
y_true shape: (10812, 101)  y_pred Shape: (10812, 101)


Calculated F-scores for cutoff: 0.2

Macro: 0.1696405054931424
Micro: 0.4614686614730878
Weighted: 0.3707221727595345
Samples: 0.45041401813233684
y_true shape: (10812, 101)  y_pred Shape: (10812, 101)


Calculated F-scores for cutoff: 0.30000000000000004

Macro: 0.12627480205396535
Micro: 0.4597470248403319
Weighted: 0.33145817307829417
Samples: 0.4555119685897118
y_true shape: (10812, 101)  y_pred Shape: (10812, 101)


Calculated F-scores for cutoff: 0.4

Macro: 0.09479958915027803
Micro: 0.4189769017203618
Weighted: 0.29511919483942306
Samples: 0.42518574846850227
y_true shape: (10812, 101)  y_pred Shape: (10812, 101)


Calculated F-scores for cutoff: 0.5

Macro: 0.06817706971746926
Micro: 0.36331962166747

# Compare to 2 Epoch Model

model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-63954-epoch-2", num_labels=124,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

predictions, raw_outputs = model.predict(eval_df['text'])

In [12]:
y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1

for cut_off in np.linspace(0.1,1,9,endpoint=False):  

    y_pred = np.empty((0,124))

    for i in range(len(raw_outputs)):
        temp = np.max(raw_outputs[i], axis=0)
        temp[temp>cut_off] = 1
        temp[temp<=cut_off] = 0
        y_pred = np.vstack([y_pred,temp])
    
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))



Calculated F-scores for cutoff: 0.1

Macro: 0.1732386101703209
Micro: 0.45128354725787634
Weighted: 0.5125896828306503
Samples: 0.4456942534521216
None:
 [0.84465825 0.83954097 0.81695389 0.65557316 0.60609388 0.42431762
 0.28475034 0.75722294 0.54137931 0.48145933 0.44047619 0.84465825
 0.60435835 0.52771392 0.59107291 0.         0.20014144 0.31687106
 0.2479564  0.6491018  0.5382193  0.60609388 0.83954097 0.
 0.13107511 0.         0.17410072 0.31193881 0.         0.
 0.29090909 0.         0.         0.         0.1641953  0.
 0.         0.         0.         0.         0.         0.14812112
 0.         0.18129703 0.11581292 0.         0.         0.
 0.         0.         0.         0.57175014 0.         0.
 0.         0.         0.         0.         0.36732068 0.
 0.         0.         0.         0.         0.38446849 0.
 0.         0.         0.41176471 0.         0.         0.
 0.         0.         0.         0.16389892 0.         0.
 0.13102687 0.         0.         0.         



Calculated F-scores for cutoff: 0.7000000000000001

Macro: 0.07082170234603283
Micro: 0.5156101381972807
Weighted: 0.3468217060142614
Samples: 0.5122318229932645
None:
 [0.90419806 0.83954097 0.81695389 0.         0.         0.
 0.         0.75722294 0.         0.         0.         0.84465825
 0.60435835 0.         0.59107291 0.         0.         0.
 0.         0.6491018  0.         0.60609388 0.83954097 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.       

# Compare to 1 Epoch Model

model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-31977-epoch-1", num_labels=124,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

predictions, raw_outputs = model.predict(eval_df['text'])

In [15]:
y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1

for cut_off in np.linspace(0.1,1,9,endpoint=False):  

    y_pred = np.empty((0,124))

    for i in range(len(raw_outputs)):
        temp = np.max(raw_outputs[i], axis=0)
        temp[temp>cut_off] = 1
        temp[temp<=cut_off] = 0
        y_pred = np.vstack([y_pred,temp])
    
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))



Calculated F-scores for cutoff: 0.1

Macro: 0.1757971450486654
Micro: 0.446947332794314
Weighted: 0.5164590768777718
Samples: 0.44343468159506666
None:
 [0.90419806 0.83954097 0.81695389 0.6550449  0.60609388 0.42431762
 0.28475034 0.75722294 0.54137931 0.48145933 0.44047619 0.84465825
 0.60435835 0.52771392 0.59107291 0.         0.20014144 0.31687106
 0.2553802  0.67919799 0.5382193  0.60609388 0.83954097 0.
 0.13107511 0.         0.17410072 0.31193881 0.         0.
 0.29090909 0.         0.08308157 0.         0.1641953  0.
 0.         0.         0.         0.         0.         0.14812112
 0.         0.18129703 0.         0.         0.         0.
 0.         0.         0.         0.57175014 0.         0.
 0.         0.         0.         0.         0.36732068 0.
 0.         0.         0.         0.         0.38446849 0.
 0.         0.         0.41176471 0.         0.         0.
 0.         0.         0.         0.16389892 0.         0.
 0.13102687 0.09741476 0.         0.         0



Calculated F-scores for cutoff: 0.7000000000000001

Macro: 0.07651056516609162
Micro: 0.5319649521337011
Weighted: 0.3661163474468814
Samples: 0.5248919290443675
None:
 [0.90419806 0.83954097 0.81695389 0.6550449  0.         0.
 0.         0.75722294 0.         0.         0.         0.84465825
 0.60435835 0.         0.59107291 0.         0.         0.
 0.         0.67919799 0.         0.60609388 0.83954097 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.       

In [50]:
sequence = '''[**2177-6-24**] 11:37 AM  CT CHEST W/O CONTRAST                                           Clip # [**Clip Number (Radiology) 90497**]
 Reason: Parenchymal disease/mass s/p evacuation of fluid in vats
 Field of view: 43
 ______________________________________________________________________________
 [**Hospital 4**] MEDICAL CONDITION:
   61 year old man with history of DOE X 5-6 weeks, with opacified hemithorax,
  thoracentesis with elevated LDH s/p VATS
 REASON FOR THIS EXAMINATION:
  Parenchymal disease/mass s/p evacuation of fluid in vats
 No contraindications for IV contrast
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  Dyspnea on exertion and pleural effusion of unknown etiology.

 TECHNIQUE:  Helical CT of the chest was performed at 5-mm collimation without
 intravenous contrast.

 COMPARISON:  CT chest of [**2177-6-22**].

 CT CHEST WITHOUT INTRAVENOUS CONTRAST:  In the interval since the prior study,
 three chest tubes have been placed in the right hemithorax.  Much of the right
 pleural effusion has been drained, with some residual locules seen
 peripherally.  The majority of the right lung has re-expanded, with some areas
 of linear atelectasis at the right base.  No focal lung mass is identified.
 The left lung is clear.  The airways are patent to the segmental bronchi
 bilaterally.  There is calcification of the coronary vessels.  The heart and
 great vessels are otherwise normal in appearance.  An enlarged right
 paratracheal lymph node is unchanged in appearance, measuring approximately 15
 mm.  There is probably an enlarged right hilar lymph node, although this is
 difficult to detect on this non-contrast study.

 In the imaged upper abdomen, a 14-mm lymph node anterior to the liver is again
 seen.  Surgical clips are seen in the right upper quadrant.  The superior
 portions of the pancreas, spleen, adrenal glands and kidneys are normal in
 appearance. Rounded thickening of the right costovertebral sulcus pleura
 extends into and past these inferior images.

 No destructive osseous lesions are identified.  There is minimal subcutaneous
 emphysema along the right chest.

 IMPRESSION:  Interval near-complete resolution of right pleural effusion, with
 some peripheral locules remaining.  No evidence of lung mass.  Several
 enlarged lymph nodes in the mediastinum persist.  An area of right posterior
 pleural thickening has an unusual rounded appearance and is incompletely
 evaluated on the current study; if there is clinical concern regarding this
 area, abdominal CT may be obtained.'''
predictions, probs = model.predict([sequence])
print(probs)

100%|██████████| 1/1 [00:00<00:00, 27.02it/s]

[[0.03378296 0.07354736 0.06561279 0.13659668 0.03396606 0.07196045
  0.08587646 0.02174377 0.05490112 0.04995728 0.06744385 0.02006531
  0.02964783 0.07366943 0.07720947 0.05041504 0.03253174 0.0526123
  0.03649902 0.03179932 0.0030632  0.35253906 0.03277588 0.0350647
  0.04077148 0.03283691 0.10778809 0.02400208 0.03314209 0.02641296
  0.02246094 0.025177   0.16687012 0.01942444 0.02763367 0.03942871
  0.16687012 0.02981567 0.0282135  0.03390503 0.10302734 0.02737427
  0.06359863 0.0970459  0.06817627 0.04360962 0.08312988 0.20349121
  0.02288818 0.0970459  0.0271759  0.02207947 0.02522278 0.02333069
  0.0526123  0.15258789 0.01945496 0.04083252 0.12731934 0.02655029
  0.02319336 0.00760651 0.00421524 0.00542831 0.00408554 0.00473785
  0.00510025 0.00605011 0.00503922 0.00443268 0.00411606 0.00448608
  0.00445175 0.00551224 0.00460815 0.00503922 0.00453949 0.04031372
  0.01872253 0.04272461 0.016922   0.03555298 0.08416748 0.0274353
  0.03079224 0.02059937 0.03378296 0.03533936 0.030

In [56]:




import pandas as pd
with open("top_codes_orderbylabel.txt", 'r') as file_handle:
    top101ICD9_list = file_handle.readlines()

print(len(top101ICD9_list))

df_map = pd.read_csv("D_ICD_DIAGNOSES_WITH_ICD10CSV.csv", sep=',')
df_map.head()


def return_topk_diagnosis_from_predictions_in_icd9(pred_tensor, topk_icd_list, topk): 
    #df_map is the mapping of all icd9, icd10 and short form and long form of diagnosis
    #topk_icd9_list is the icd9 codes of top 100 diseases and "Other" in mimic 3
    #pred_array is the array (tensor) of probabilities of diseases. shape (1,150)
    #assert()
    #if(type(pred_tensor) != 'numpy.ndarray'):
    #    tmp = pred_tensor.numpy().reshape(-1)
    #else:
    tmp = pred_tensor.reshape(-1)
    #print(type(tmp))
    tmp = np.argpartition(tmp, -topk )[-topk:]#a.numpy() converts the tensor to numpy array
    
    topkICDPreds = []
    for idx in tmp:
        icd =  topk_icd_list[idx].strip()
    #print(icd)
        topkICDPreds.append(icd)
    
    return topkICDPreds   #return a List of topk  icd 9 codes
    
    # If the probability is same the icd9 method and text method may create erronous results. 
    # a better solutions is to return icd 9 and text description together.
def return_topk_diagnosis_from_predictions_in_Text(pred_tensor,topk_icd_list, topk, df_map): 
    #df_map is the mapping of all icd9, icd10 and short form and long form of diagnosis
    #topk_icd9_list is the icd9 codes of top 101 diseases in mimic 3
    #pred_array is the array (tensor) of probabilities of diseases. shape (1,150)
    topkICDPreds = return_topk_diagnosis_from_predictions_in_icd9(pred_tensor, topk_icd_list,topk)  
    predictions = df_map.loc[df_map['ICD9_CODE'].isin(topkICDPreds)]
    print(predictions)
    #print(predictions)
    text_preds = predictions['LONG_TITLE'].tolist()
    if "OTHER" in topkICDPreds:
        idx = topkICDPreds.index("OTHER")
        text_preds.insert(idx,"OTHER")
    #text_preds = [s.strip() for s in text_preds]
    return text_preds

def return_topk_diagnosis_from_predictions_with_probability (pred_tensor,topk_icd_list, topk, df_map): 
    #df_map is the mapping of all icd9, icd10 and short form and long form of diagnosis
    #topk_icd9_list is the icd9 codes of top 101 diseases in mimic 3
    #pred_array is the array (tensor) of probabilities of diseases. shape (1,150)
    topkICDPreds = return_topk_diagnosis_from_predictions_in_icd9(pred_tensor, topk_icd_list, topk)
    text_preds = return_topk_diagnosis_from_predictions_in_Text(pred_tensor, topk_icd_list, topk, df_map)
    
    probs = pred_tensor.reshape(-1)
    tmp = np.argpartition(probs, -topk )[-topk:]    #a.numpy() converts the tensor to numpy array
    topkprobs = []  # to store the topk highest probabilities
    #topkICDPreds = {}
    for idx in tmp:
        topkprobs.append(probs[idx])
        
    prob_dict = dict(zip(topkICDPreds,topkprobs))
    text_dict = dict(zip(topkICDPreds,text_preds))
    
    if 'OTHER' in topkICDPreds:
        text_dict['OTHER'] = "Any Other Disease Not Managed by Current Version"    
    return prob_dict, text_dict

def return_actual_diagnoses (y_true, topk_icd_list, df_map):
    tmp = np.where(np.stack(seq2_actpred)==1)
    tmp = list(tmp[0])  # reads the list containing all elements of tuple
    topkICDPreds = []
    for i in range(len(tmp)):
        icd =  topk_icd_list[i].strip()
        topkICDPreds.append(icd)
    
    predictions = df_map.loc[df_map['ICD9_CODE'].isin(topkICDPreds)]
    text_preds = predictions['LONG_TITLE'].tolist()
    
    text_dict = dict(zip(topkICDPreds,text_preds))
    return text_dict

101


In [116]:
def return_topk_diagnosis_from_predictions(pred_tensor,topk_icd_list,df_map,**kwargs):
    # topk, return_prob, return_diagnosis       
    if not 'topk' in kwargs.keys():
        topk=5
    else:
        topk= kwargs['topk']

    if 'return_prob' in kwargs.keys():
        return_prob =  kwargs['return_prob']
    else:
        return_prob = False     
        
    if 'return_diagnosis' in kwargs.keys():
        return_diagnosis =  kwargs['return_diagnosis']
    else:
        return_diagnosis = False
              
    #print(f"Parameters: topk:{topk}  return_prob:{return_prob}  return_diagnosis:{return_diagnosis}")

    if ( not return_prob and  not return_diagnosis):
        print(f" 1 activated")
        tmp = pred_tensor.reshape(-1)
        tmp = np.argpartition(tmp, -topk )[-topk:]#a.numpy() converts the tensor to numpy array
        topkICDPreds = []
        for idx in tmp:
            icd =  topk_icd_list[idx].strip()
    #print(icd)
            topkICDPreds.append(icd)
        return topkICDPreds
        #return return_topk_diagnosis_from_predictions_in_icd9(pred_tensor, topk_icd_list, topk)
    elif  (return_prob and not return_diagnosis):
        print(f" 2 activated")
       # topkICDPreds = return_topk_diagnosis_from_predictions_in_icd9(pred_tensor, topk_icd_list, topk)
        prob = pred_tensor.reshape(-1)
        indices = np.argpartition(prob, -topk )[-topk:]#a.numpy() converts the tensor to numpy array
        #print(indices)
        topkICDPreds = []
        topkprobs = []  # to store the topk highest probabilities
        for idx in indices:
            icd =  topk_icd_list[idx].strip()
            topkICDPreds.append(icd)
            topkprobs.append(prob[idx])   
        #print(topkICDPreds)
        #print(topkprobs)
        prob_dict = dict(zip(topkICDPreds,topkprobs))
        return prob_dict
    elif  ( not return_prob and return_diagnosis):
        print(f" 3 activated")
        prob = pred_tensor.reshape(-1)
        indices = np.argpartition(prob, -topk )[-topk:]#a.numpy() converts the tensor to numpy array
        topkICDPreds = []
        for idx in indices:
            icd =  topk_icd_list[idx].strip()
            topkICDPreds.append(icd)
        try:
            predictions = df_map.loc[df_map['ICD9_CODE'].isin(topkICDPreds)]
            text_preds = predictions['LONG_TITLE'].tolist()
            if "OTHER" in topkICDPreds:
                idx = topkICDPreds.index("OTHER")
                text_preds.insert(idx,"OTHER")
            text_dict = dict(zip(topkICDPreds,text_preds))
            return text_dict
        except:
            print (f"exception occurred in diagnoses calculation.")
            return
    elif (return_prob and return_diagnosis):
        print(f" 4 activated")
        prob = pred_tensor.reshape(-1)
        indices = np.argpartition(prob, -topk )[-topk:]#a.numpy() converts the tensor to numpy array
        topkICDPreds = []
        topkprobs = []  # to store the topk highest probabilities
        for idx in indices:
            icd =  topk_icd_list[idx].strip()
            topkICDPreds.append(icd)
            topkprobs.append(prob[idx])
        try:
            predictions = df_map.loc[df_map['ICD9_CODE'].isin(topkICDPreds)]
            text_preds = predictions['LONG_TITLE'].tolist()
            if "OTHER" in topkICDPreds:
                idx = topkICDPreds.index("OTHER")
                text_preds.insert(idx,"OTHER")
            text_dict = dict(zip(topkICDPreds,text_preds))
            prob_dict = dict(zip(topkICDPreds,topkprobs))
            return prob_dict, text_dict
        except:
            print (f"exception occurred in diagnoses calculation.")
            return

tmp = np.where(np.stack(seq2_actpred)==1)
#print(tmp)
tmp = list(tmp[0])
topkICDPreds = []
for i in range(len(tmp)):
    #print(tmp[i])
    icd =  top101ICD9_list[i].strip()
    topkICDPreds.append(icd)
print(topkICDPreds)
predictions = df_map.loc[df_map['ICD9_CODE'].isin(topkICDPreds)]
print(predictions.head())

In [117]:
#print(return_topk_diagnosis_from_predictions_in_icd9(probs,top101ICD9_list,5))
#print(return_topk_diagnosis_from_predictions_in_Text(probs,top101ICD9_list,5, df_map))
#prob_dict,text_dict = return_topk_diagnosis_from_predictions(probs,top101ICD9_list,df_map, topk=7, return_prob=True, return_diagnosis=True)
text_dict = return_topk_diagnosis_from_predictions(probs,top101ICD9_list,df_map, topk=7, return_prob=False, return_diagnosis=False)
#print(prob_dict)
print(text_dict)

 1 activated
['25000', '5849', '4280', '42731', '51881', '4019', 'OTHER']


In [55]:
prob_dict, text_dict = return_topk_diagnosis_from_predictions_with_probability(probs,top101ICD9_list,5, df_map)
for key in prob_dict.keys():
    print(f"ICD9_code: {key}  Diagnoses:{text_dict[key]}  Probability:{prob_dict[key]}")

ICD9_code: 4280  Diagnoses:Acute respiratory failure  Probability:0.1668701171875
ICD9_code: 42731  Diagnoses:Congestive heart failure, unspecified  Probability:0.1668701171875
ICD9_code: 51881  Diagnoses:Atrial fibrillation  Probability:0.2034912109375
ICD9_code: 4019  Diagnoses:Unspecified essential hypertension  Probability:0.3525390625
ICD9_code: OTHER  Diagnoses:Any Other Disease Not Managed by Current Version  Probability:0.9951171875


In [26]:

sequence2 =''' "Nursing Progress Note 0700-1900

Events:  Given 1.5 mg ativan prior to MRI, unable to obtain good scan D/T anxiety & agitation.  Repeat NA at noon 119

Neuro:  A&Ox2, unsure of date.  MAE, PERRLA.  Remains on CIWA scale scores less than 5, but remains agitated at times. Given 2.5 mg PO valium at noon with little effect.  Remains in C-collar until MRI can clear her C-spine.

Resp:  Lung sounds clear diminished at bases, RR mid teens sats 95-97 on RA.  Respirations even and unlabored.  + non-productive cough.

Cardiac:  Tele SR 70's without ectopy.  Hemodynamically stable.  + 3 pt/dp, no edema

GI:  PO intake very poor, refusing meals, drinking ensures TID.  + BS in 4 quadrents, abdomen soft non-distended,  No BM since admission on bowel regimen

Renal:  Foley D/C'd at noon DTV.  UOP has been marginal.  Urine dark and amber in color

ID:  afebrile on Flagyl/ceftriaxone with PNA.

Skin:  Intact no current issues

FEN:  NA at noon 119 remains on 1.5 liter fluid restriction.  repeat NA due at [**2162**]

Plan:

1.  C/O to floor
2.  Monitor neuro status, C-collar repeat MRI in AM
3.  Serial NA checks fluid restrict
4.  IV antibiotics as ordered
5.  Routine monitoring and care'''
predictions, probs2 = model.predict([sequence2])

100%|██████████| 1/1 [00:00<00:00, 25.63it/s]

['4280', 'OTHER', '42731', '4019', '51881']
['Acute respiratory failure', 'Congestive heart failure, unspecified', 'Atrial fibrillation', 'Unspecified essential hypertension']
ICD9_code: 4280  Diagnoses:Acute respiratory failure  Probability:0.10577392578125
ICD9_code: OTHER  Diagnoses:Any Other Disease Not Managed by Current Version  Probability:0.99560546875
ICD9_code: 42731  Diagnoses:Atrial fibrillation  Probability:0.11260986328125
ICD9_code: 4019  Diagnoses:Unspecified essential hypertension  Probability:0.32177734375


KeyError: '51881'

In [ ]:
prob_dict, text_dict = return_topk_diagnosis_from_predictions_with_probability(probs2,top101ICD9_list,5, df_map)
print(return_topk_diagnosis_from_predictions_in_icd9(probs2,top101ICD9_list,5))
print(return_topk_diagnosis_from_predictions_in_Text(probs2,top101ICD9_list,5,df_map))
for key in prob_dict.keys():
    print(f"ICD9_code: {key}  Diagnoses:{text_dict[key]}  Probability:{prob_dict[key]}")

In [64]:
seq2_actpred= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
text_dict = return_actual_diagnoses (seq2_actpred, top101ICD9_list,df_map)
for key in text_dict.keys():
    print(f"ICD9_code: {key}  Diagnoses:{text_dict[key]}")
    

ICD9_code: 01193  Diagnoses:Other primary cardiomyopathies
